# Convoluted Neural Network: 1st Model Trial

## Imports

In [18]:
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

## Dataframe

In [15]:
import pickle
pickle_path = ('../raw_data/clean.pickle')
with open(pickle_path, 'rb') as f:
    df = pickle.load(f)

In [16]:
df.head(3)

,precinct_number,offense_type,crime_completed,offense_level,premise_desc,premise,jurisdiction,park_name,suspect_age,suspect_race,suspect_sex,latitude,longitude,metro,victim_age,victim_race,victim_sex,period,borough,patrol_borough
0,73,OFFENSES AGAINST PUBLIC ADMINI,COMPLETED,MISDEMEANOR,INSIDE,RESIDENCE - PUBLIC HOUSING,N.Y. HOUSING POLICE,NOT PARK,25-44,BLACK,M,True,True,NOT SUBWAY,<18,BLACK,M,2015-02-11 15:00:00,BROOKLYN,PATROL BORO BKLYN NORTH
1,69,ASSAULT 3 & RELATED OFFENSES,COMPLETED,MISDEMEANOR,INSIDE,RESIDENCE-HOUSE,N.Y. POLICE DEPT,NOT PARK,UNKNOWN,UNKNOWN,UNKNOWN,True,True,NOT SUBWAY,45-64,BLACK,F,2012-03-17 10:00:00,BROOKLYN,PATROL BORO BKLYN SOUTH
2,71,HARRASSMENT 2,COMPLETED,VIOLATION,INSIDE,PUBLIC SCHOOL,N.Y. POLICE DEPT,NOT PARK,<18,BLACK,M,True,True,NOT SUBWAY,18-24,BLACK,M,2016-10-27 13:00:00,BROOKLYN,PATROL BORO BKLYN SOUTH


## Input Prep

### Train and Test df

#### Sort dataframe by period !

First df needs to be sorted by period!

In [23]:
sorted_df = df.sort_values(by = ['period'])

In [24]:
sorted_df.head(3)

,precinct_number,offense_type,crime_completed,offense_level,premise_desc,premise,jurisdiction,park_name,suspect_age,suspect_race,suspect_sex,latitude,longitude,metro,victim_age,victim_race,victim_sex,period,borough,patrol_borough
303902,46,THEFT-FRAUD,COMPLETED,FELONY,INSIDE,RESIDENCE - APT. HOUSE,N.Y. POLICE DEPT,NOT PARK,UNKNOWN,UNKNOWN,UNKNOWN,True,True,NOT SUBWAY,25-44,BLACK,M,2007-01-01 00:00:00,BRONX,PATROL BORO BRONX
5119621,33,FRAUDS,COMPLETED,MISDEMEANOR,INSIDE,RESIDENCE - APT. HOUSE,N.Y. POLICE DEPT,NOT PARK,UNKNOWN,UNKNOWN,UNKNOWN,True,True,NOT SUBWAY,45-64,WHITE HISPANIC,F,2007-01-01 00:00:00,MANHATTAN,PATROL BORO MAN NORTH
5410933,73,SEX CRIMES,COMPLETED,FELONY,INSIDE,RESIDENCE - APT. HOUSE,N.Y. POLICE DEPT,NOT PARK,UNKNOWN,UNKNOWN,M,True,True,NOT SUBWAY,<18,BLACK,M,2007-01-01 00:00:00,BROOKLYN,PATROL BORO BKLYN NORTH


#### Train & Test Dataframe

In [25]:
len_ = int(0.8 * sorted_df.shape[0])

len_

5132909

In [26]:
df_train = sorted_df[:len_]
df_test = sorted_df[len_:]

#### Subsample for multiple independent datasequences to train on:

In [ ]:
def subsample_sequence(df, length):
    
    last_possible = df.shape[0] - length
    
    random_start = np.random.randint(0, last_possible)
    df_sample = df[random_start: random_start+length]
    
    return df_sample

df_subsample = subsample_sequence(df, 10)

In [ ]:
def split_subsample_sequence(df, length):
              
    df_subsample = subsample_sequence(df, length)
    y_sample = df_subsample.iloc[df_subsample.shape[0]-1]['pm2.5']
    
    X_sample = df_subsample[0:df_subsample.shape[0]-1]
    X_sample = X_sample.fillna(fillna(X_sample, df_mean))
    X_sample = X_sample.values
    
    return np.array(X_sample), np.array(y_sample)

In [ ]:
def get_X_y(df, number_of_sequences, length):
    X, y = [], []

    for i in range(number_of_sequences):
        xi, yi = split_subsample_sequence(df, length)
        X.append(xi)
        y.append(yi)
        
    X = np.array(X)
    y = np.array(y)
        
    return X, y

In [ ]:
X_train, y_train = get_X_y(df_train, 1000, 51)
X_test, y_test = get_X_y(df_test, 200, 51)

### X and y

In [2]:
X = np.random.randint(2, size = (50, 24, 1_000, 1_000))

In [3]:
y = np.random.randint(2, size = (50, 3, 1_000, 1_000))

In [12]:
print(X.shape)
print(y.shape)

(50, 24, 1000, 1000)
(50, 3, 1000, 1000)


### Train, test, split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

## CNN

In [4]:
# add gaussian filter # mse

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.callbacks import EarlyStopping